In [1]:
%load_ext autoreload
%autoreload 2

%load_ext lab_black

In [2]:
import sys
from pathlib import Path

In [3]:
_pwd = Path(".").resolve()
print(_pwd)
sys.path.append(str(_pwd))

/data/ik18445_cache/projects/phenotype-mapping/analysis/notebooks/mvp_ontology_distance


In [4]:
import pandas as pd
import janitor
from pydash import py_

In [5]:
from common_funcs import utils

In [8]:
proj_root = utils.find_project_root("docker-compose.yml")
assert proj_root.exists(), proj_root

data_dir = proj_root / "data"
input_data_path = (
    data_dir / "mvp-efo-terms-2022-12" / "efo_all_sources_wide_19DEC2022.csv"
)
assert input_data_path.exists(), input_data_path

In [9]:
raw_df = pd.read_csv(input_data_path)
print(raw_df.info())
raw_df

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11109 entries, 0 to 11108
Data columns (total 12 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   EFO_ID             11109 non-null  object
 1   EFO_Term           11109 non-null  object
 2   Adverse_events     11109 non-null  int64 
 3   Box_warnings       11109 non-null  int64 
 4   Drug_indications   11109 non-null  int64 
 5   FinnGen            11109 non-null  int64 
 6   GWAS_Catalog       11109 non-null  int64 
 7   MVP/UKBB           11109 non-null  int64 
 8   OMIM               11109 non-null  int64 
 9   Safety_literature  11109 non-null  int64 
 10  adverse_events     11109 non-null  int64 
 11  pLOF               11109 non-null  int64 
dtypes: int64(10), object(2)
memory usage: 1.0+ MB
None


,EFO_ID,EFO_Term,Adverse_events,Box_warnings,Drug_indications,FinnGen,GWAS_Catalog,MVP/UKBB,OMIM,Safety_literature,adverse_events,pLOF
0,CHEBI_16737,creatinine,0,0,0,0,0,1,0,0,0,0
1,CHEBI_16990,bilirubin,0,0,0,0,0,1,0,0,0,0
2,CHEBI_17347,testosterone,0,0,0,0,0,1,0,0,0,0
3,CHEBI_23965,estradiol,0,0,0,0,0,1,0,0,0,0
4,CHEBI_26020,phosphate,0,0,0,0,0,1,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...
11104,UBERON_0000947,aorta,0,0,0,1,0,0,0,0,0,0
11105,UBERON_0001467,shoulder,0,0,0,1,0,0,0,0,0,0
11106,UBERON_0002198,neurohypophysis,0,0,0,0,0,1,0,0,0,0
11107,UBERON_0002384,connective tissue,0,0,0,1,0,0,0,0,0,0


In [11]:
raw_df["EFO_ID"].drop_duplicates().__len__()

11091

In [12]:
raw_df[["EFO_ID", "EFO_Term"]].drop_duplicates().__len__()

11109

In [13]:
raw_df[["EFO_Term"]].drop_duplicates().__len__()

10864

In [17]:
dupes = raw_df[raw_df.duplicated(["EFO_ID"], keep=False)]
dupes

,EFO_ID,EFO_Term,Adverse_events,Box_warnings,Drug_indications,FinnGen,GWAS_Catalog,MVP/UKBB,OMIM,Safety_literature,adverse_events,pLOF
68,EFO_0000319,cardiovascular disease,1,1,1,1,1,1,0,0,0,1
69,EFO_0000319,cardiovascular system diseaes,0,0,0,0,0,1,0,0,0,0
143,EFO_0000618,nervous system disease,1,1,1,1,1,1,0,0,0,0
144,EFO_0000618,nervous sytem disease,0,0,0,0,0,1,0,0,0,0
266,EFO_0002506,obsolete_osteoarthritis,0,0,0,1,0,0,0,0,0,0
267,EFO_0002506,osteoarthritis,0,0,1,0,0,0,0,0,0,0
390,EFO_0003847,intellectual disability,0,0,0,1,0,0,0,0,0,0
391,EFO_0003847,mental retardation,1,0,1,0,0,0,0,0,1,0
1035,EFO_0005297,granulomatosis with polyangiitis,1,0,1,1,1,0,0,0,0,0
1036,EFO_0005297,granulomatosis with polyangitis,0,0,0,0,0,1,0,0,0,0


In [18]:
dupes.to_csv("duplicated.csv", index=True)